In [1]:
import pandas as pd
import networkx as nx
import os
from time import time

from networkx.algorithms.centrality import edge_betweenness_centrality
from networkx.algorithms.components import connected_components
from networkx.algorithms.community.quality import modularity
from networkx.algorithms.community.quality import partition_quality
from networkx.algorithms.community.quality import performance

In [2]:
def load_data(filename):
    input_data = pd.read_csv(filename, header=None)
    G = nx.Graph(input_data.values)
    return G

In [3]:
def girvan_newman(G, k=None, scoring=modularity):
    g = G.copy()
    answers = [list(connected_components(g))]
    while len(g.edges):
        best_edge = None
        best_val = -float('inf')
        for edge, val in edge_betweenness_centrality(g).items():
            if val > best_val:
                best_val = val
                best_edge = edge
        g.remove_edge(*best_edge)
        components = list(connected_components(g))
        if len(components) != len(answers[-1]):
            answers.append(components)
        if k is not None and len(components) == k:
            return components
        
    best_i = None
    best_val = -float('inf')
    for i, comm  in enumerate(answers):
        v = scoring(G, comm)
        if v > best_val:
            best_val = v
            best_i = i
    return answers[best_i]

In [4]:
def coloring(g, communities):
    color_map = []
    colors = list(range(len(communities)))
    for node in g:
        for i, com in enumerate(communities):
            if node in com:
                color_map.append(i+1)
    return color_map

In [5]:
def save_output(filename, g, communities):
    df = pd.DataFrame(coloring(g, communities))
    df.index += 1
    df.to_csv('out_'+filename, header=False)

In [6]:
dir_ = "competition/"

networks = []
for file in os.listdir(dir_):
    if file.endswith(".csv"):
        filename = dir_ + file
        print(filename)
        k = None
        if 'K=' in file:
            k = int(file[file.index('=')+1:file.index('.csv')])
        networks.append((filename, k))

competition/D1-K=2.csv
competition/D3-UNC.csv
competition/D3-K=12.csv
competition/D2-UNC.csv
competition/D2-K=7.csv
competition/D1-UNC.csv


In [7]:
for filename, k in networks:
    g = load_data(filename)
    start = time()
    communities = girvan_newman(g, k=k)
    running_time = time() - start
    print(f"{{{filename[filename.index('/')+1:]}, {running_time:.3f}}}", end=', ')
    save_output(filename, g, communities)

{D1-K=2.csv, 0.032}, {D3-UNC.csv, 2.374}, {D3-K=12.csv, 3.473}, {D2-UNC.csv, 11.182}, {D2-K=7.csv, 0.163}, {D1-UNC.csv, 1.631}, 